In [1]:
! kaggle competitions download -c spaceship-titanic

spaceship-titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
! unzip spaceship-titanic.zip -d ./spaceship

Archive:  spaceship-titanic.zip
  inflating: ./spaceship/sample_submission.csv  
  inflating: ./spaceship/test.csv    
  inflating: ./spaceship/train.csv   


In [14]:
import pandas as pd
import numpy as np

filepath_train = '../Kaggle_projects/spaceship/train.csv'
filepath_test = '../Kaggle_projects/spaceship/test.csv'


train_df = pd.read_csv(filepath_train)
test_df = pd.read_csv(filepath_test)

In [15]:
def combine_df(train_df,test_df):
    return pd.concat([train_df,test_df],ignore_index=True)


def separate_df(combined_df, train_df_len, test_df_len):
    train_df = combined_df.iloc[:train_df_len]
    test_df = combined_df.iloc[train_df_len: test_df_len+train_df_len]
    return train_df, test_df

full_df = combine_df(train_df,test_df)

In [16]:
full_df = full_df.drop(['PassengerId','Name', 'Transported'],axis=1)

cat_cols = []
num_cols = []

for cols in full_df.select_dtypes(include='object').columns:
    cat_cols.append(cols)

for cols in full_df._get_numeric_data().columns:
    num_cols.append(cols)

print(cat_cols)
num_cols

['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']


['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [17]:
full_df.isnull().sum()

HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
dtype: int64

In [18]:
# Filling Cat variables with the most common
for cols in cat_cols:
    full_df[cols].fillna(full_df[cols].mode()[0], inplace=True)

In [19]:
# Filling Num variables with the average

for num in num_cols:
    full_df[num].fillna(full_df[num].mean(), inplace=True)

In [20]:
#Get more information from Cabin
cabins = full_df['Cabin'].str.split(pat='/', expand=True)
cabins = cabins.rename(columns={0: 'Deck', 1:'Number', 2:'Side'})
full_df = pd.concat([full_df, cabins], axis=1)
full_df = full_df.drop(labels=['Cabin'], axis=1)
full_df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Number,Side
0,Europa,False,TRAPPIST-1e,39.000000,False,0.0,0.0,0.0,0.0,0.0,B,0,P
1,Earth,False,TRAPPIST-1e,24.000000,False,109.0,9.0,25.0,549.0,44.0,F,0,S
2,Europa,False,TRAPPIST-1e,58.000000,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S
3,Europa,False,TRAPPIST-1e,33.000000,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S
4,Earth,False,TRAPPIST-1e,16.000000,False,303.0,70.0,151.0,565.0,2.0,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,Earth,True,TRAPPIST-1e,34.000000,False,0.0,0.0,0.0,0.0,0.0,G,1496,S
12966,Earth,False,TRAPPIST-1e,42.000000,False,0.0,847.0,17.0,10.0,144.0,G,160,P
12967,Mars,True,55 Cancri e,28.771969,False,0.0,0.0,0.0,0.0,0.0,D,296,P
12968,Europa,False,TRAPPIST-1e,28.771969,False,0.0,2680.0,0.0,0.0,523.0,D,297,P


In [21]:
full_df.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Deck            0
Number          0
Side            0
dtype: int64

In [22]:
full_df.describe(include='all')

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Number,Side
count,12970,12970,12970,12970.000000,12970,12970.000000,12970.000000,12970.000000,12970.000000,12970.000000,12970,12970,12970
unique,3,2,3,NaN,2,NaN,NaN,NaN,NaN,NaN,8,1894,2
top,Earth,False,TRAPPIST-1e,NaN,False,NaN,NaN,NaN,NaN,NaN,F,160,P
freq,7153,8389,9145,NaN,12697,NaN,NaN,NaN,NaN,NaN,4239,318,6589
mean,NaN,NaN,NaN,28.771969,NaN,222.897852,451.961675,174.906033,308.476904,306.789482,NaN,NaN,NaN
std,NaN,NaN,NaN,14.236710,NaN,640.996685,1566.618306,583.550067,1117.835494,1167.840416,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,20.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,37.000000,NaN,79.000000,126.000000,47.000000,87.000000,64.000000,NaN,NaN,NaN


In [23]:
print(len(train_df))
print(len(test_df))

8693
4277


In [33]:
# Separating train and test df
train_df, test_df = separate_df(combined_df=full_df,train_df_len=8693,test_df_len=4277)

# Reattaching the target feature
original_train_df = pd.read_csv(filepath_train)
train_df.loc[:,'Transported'] = original_train_df['Transported']

train_df.head()


C:\Users\mehdi\AppData\Local\Temp\ipykernel_18196\2746190217.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:,'Transported'] = original_train_df['Transported']


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Number,Side,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0,P,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0,S,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1,S,True


Modeling:

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


# Loading X,y
X = train_df.drop('Transported',axis=1)
y = train_df['Transported']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.33, random_state=42)

# Identify numeric columns
numeric_columns = X_train.select_dtypes(include=[np.number]).columns

# Create a ColumnTransformer to apply the StandardScaler only to numeric columns
preprocessor = ColumnTransformer(
    transformers=[('num', StandardScaler(), numeric_columns)])

# Defining models
models = [
    ("Random Forest", RandomForestClassifier()),
    ("Gradient Boosting", GradientBoostingClassifier()),
    ("SVM", SVC()),
    ("Logistic Regression", LogisticRegression())
]

# Creating a pipline for evaluating different models
results = []
for name, model in models:
    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Evaluate the model using cross-validation
    scores = cross_val_score(model_pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    results.append((name, scores.mean(), scores.std()))

# Sort the results based on accuracy
results.sort(key=lambda x: x[1], reverse=True)

for name, mean_score, std_score in results:
    print(f"{name}: Mean Accuracy = {mean_score:.2f}, Std = {std_score:.2f}")



Gradient Boosting: Mean Accuracy = 0.79, Std = 0.01
SVM: Mean Accuracy = 0.79, Std = 0.01
Random Forest: Mean Accuracy = 0.78, Std = 0.01
Logistic Regression: Mean Accuracy = 0.77, Std = 0.01


In [40]:
from sklearn.model_selection import GridSearchCV

# Define the Gradient Boosting parameter grid
param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 4, 5],
    'model__min_samples_split': [2, 3, 4],
    'model__min_samples_leaf': [1, 2, 3]
}

# Create a pipeline with the selected model (Gradient Boosting) and preprocessing steps
model = GradientBoostingClassifier()  # Specify the model
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Create the grid search
grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("Best Parameters for Gradient Boosting:")
print(best_params)
print("Best Accuracy:", best_accuracy)


Best Parameters for Gradient Boosting:
{'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 150}
Best Accuracy: 0.7943005471734288


In [43]:
# Finalizing model

final_model = GradientBoostingClassifier(learning_rate=0.1, max_depth=3,min_samples_leaf=2, min_samples_split=2, n_estimators=150)

pipe_model = Pipeline([('preprocessor', preprocessor),('final_model', final_model)])

pipe_model.fit(X_train,y_train)

# Performance

preds = pipe_model.predict(X_test)
accuracy_score(y_true=y_test,y_pred=preds)


0.7765772046009063

In [45]:
# Predictions

predictions = pipe_model.predict(test_df)

In [73]:
df_submissions = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Transported':predictions})
df_submissions.to_csv('./spaceship/submission.csv',index=False)